# 1. Preprocessing

Ce notebook gère le preprocessing complet des données du fichier jobs.csv

## Import des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("Bibliothèques importées avec succès")

Bibliothèques importées avec succès


## Chargement des données

In [ ]:
# Charger le fichier CSV avec le bon séparateur
df = pd.read_csv('../data/csv/jobs.csv', sep=';', encoding='utf-8')

print(f"Shape des données: {df.shape}")
print(f"\nColonnes: {df.columns.tolist()}")
print(f"\nPremières lignes:")
df.head()

Shape des données: (2458, 4)

Colonnes: ['Job Title', 'Skills', 'Job Description', 'Certifications']

Premières lignes:


,Job Title,Skills,Job Description,Certifications
0,Software Engineer,Java;Spring Boot;MySQL;Git;REST APIs,Conçoit et développe des applications backend ...,Oracle Certified Professional Java SE;AWS Cert...
1,Software Engineer,Python;Django;PostgreSQL;Git;REST APIs,Developpe des services web evolutifs et assure...,PCAP - Certified Associate in Python Programmi...
2,Software Engineer,C#;.NET Core;SQL Server;Azure DevOps;OOP,Participe à la conception et au développement ...,Microsoft Certified: Azure Developer Associate
3,Software Engineer,Java;Hibernate;Maven;JUnit;Git,Implémente des fonctionnalités backend et écri...,Oracle Certified Associate Java SE
4,Software Engineer,Python;Flask;SQLite;Docker;Git,Crée des microservices légers et maintenables ...,Docker Certified Associate


## Exploration des données

In [4]:
# Informations sur les données
print("Informations sur le dataset:")
print(df.info())

print("\n" + "="*50)
print("Statistiques descriptives:")
print(df.describe())

Informations sur le dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458 entries, 0 to 2457
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Job Title        2458 non-null   object
 1   Skills           2458 non-null   object
 2   Job Description  2458 non-null   object
 3   Certifications   2458 non-null   object
dtypes: object(4)
memory usage: 76.9+ KB
None

Statistiques descriptives:
                Job Title                                             Skills  \
count                2458                                               2458   
unique                119                                               2225   
top     Backend Developer  Administrative Support;Document Management;Sch...   
freq                   45                                                  9   

                                         Job Description  \
count                                               2458   


In [5]:
# Vérifier les valeurs manquantes
print("Valeurs manquantes par colonne:")
print(df.isnull().sum())

print("\n" + "="*50)
print(f"Pourcentage de valeurs manquantes:")
print((df.isnull().sum() / len(df)) * 100)

Valeurs manquantes par colonne:
Job Title          0
Skills             0
Job Description    0
Certifications     0
dtype: int64

Pourcentage de valeurs manquantes:
Job Title          0.0
Skills             0.0
Job Description    0.0
Certifications     0.0
dtype: float64


In [6]:
# Distribution de la variable cible (Job Title)
print("Distribution des Job Titles:")
print(df['Job Title'].value_counts())

print("\n" + "="*50)
print(f"Nombre de classes uniques: {df['Job Title'].nunique()}")

Distribution des Job Titles:
Job Title
Backend Developer            45
Software Engineer            44
Machine Learning Engineer    44
Cybersecurity Analyst        44
Blockchain Developer         43
                             ..
Teacher                      10
Academic Coordinator         10
Instructor                   10
Training Specialist          10
Job Title                     1
Name: count, Length: 119, dtype: int64

Nombre de classes uniques: 119


## Nettoyage des données

In [7]:
# Créer une copie pour le preprocessing
df_clean = df.copy()

# Supprimer les lignes avec des valeurs manquantes dans la variable cible
print(f"Lignes avant suppression des NaN dans Job Title: {len(df_clean)}")
df_clean = df_clean[df_clean['Job Title'].notna()]
print(f"Lignes après suppression: {len(df_clean)}")

Lignes avant suppression des NaN dans Job Title: 2458
Lignes après suppression: 2458


In [8]:
# Remplir les valeurs manquantes dans les features avec une chaîne vide
text_columns = ['Skills', 'Job Description', 'Certifications']
for col in text_columns:
    df_clean[col] = df_clean[col].fillna('')

print("Valeurs manquantes après traitement:")
print(df_clean.isnull().sum())

Valeurs manquantes après traitement:
Job Title          0
Skills             0
Job Description    0
Certifications     0
dtype: int64


In [9]:
# Fonction de nettoyage de texte
def clean_text(text):
    """
    Nettoie le texte en:
    - Convertissant en minuscules
    - Supprimant les caractères spéciaux sauf espaces et point-virgules
    - Normalisant les espaces
    """
    if not isinstance(text, str):
        return ''
    
    # Convertir en minuscules
    text = text.lower()
    
    # Remplacer les point-virgules par des espaces
    text = text.replace(';', ' ')
    
    # Supprimer les caractères spéciaux mais garder les lettres accentuées
    text = re.sub(r'[^a-zàâäéèêëïîôöùûüÿçæœ0-9\s]', ' ', text)
    
    # Normaliser les espaces multiples
    text = re.sub(r'\s+', ' ', text)
    
    # Supprimer les espaces en début et fin
    text = text.strip()
    
    return text

print("Fonction de nettoyage définie")

Fonction de nettoyage définie


In [10]:
# Appliquer le nettoyage aux colonnes textuelles
print("Nettoyage des colonnes textuelles...")

for col in text_columns:
    print(f"Nettoyage de {col}...")
    df_clean[col] = df_clean[col].apply(clean_text)

print("\nNettoyage terminé!")
print("\nExemple après nettoyage:")
df_clean[text_columns].head()

Nettoyage des colonnes textuelles...
Nettoyage de Skills...
Nettoyage de Job Description...
Nettoyage de Certifications...

Nettoyage terminé!

Exemple après nettoyage:


,Skills,Job Description,Certifications
0,java spring boot mysql git rest apis,conçoit et développe des applications backend ...,oracle certified professional java se aws cert...
1,python django postgresql git rest apis,developpe des services web evolutifs et assure...,pcap certified associate in python programming...
2,c net core sql server azure devops oop,participe à la conception et au développement ...,microsoft certified azure developer associate
3,java hibernate maven junit git,implémente des fonctionnalités backend et écri...,oracle certified associate java se
4,python flask sqlite docker git,crée des microservices légers et maintenables ...,docker certified associate


## Encodage de la variable cible

In [11]:
# Encoder la variable cible
label_encoder = LabelEncoder()
df_clean['Job Title Encoded'] = label_encoder.fit_transform(df_clean['Job Title'])

print(f"Nombre de classes: {len(label_encoder.classes_)}")
print(f"\nExemples d'encodage:")
print(df_clean[['Job Title', 'Job Title Encoded']].drop_duplicates().head(10))

Nombre de classes: 119

Exemples d'encodage:
                    Job Title  Job Title Encoded
0           Software Engineer                101
44   Senior Software Engineer                 97
84         Software Developer                100
127      Full Stack Developer                 46
168         Backend Developer                  9
213        Frontend Developer                 45
254             Web Developer                116
294       WordPress Developer                118
335          Mobile Developer                 67
378   Game Developer (Intern)                 47


In [ ]:
# Sauvegarder le label encoder pour une utilisation ultérieure
with open('../data/pkl/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("Label encoder sauvegardé: ../data/pkl/label_encoder.pkl")

Label encoder sauvegardé: label_encoder.pkl


## Combinaison des features textuelles

In [13]:
# Combiner toutes les colonnes textuelles en une seule feature
df_clean['Combined_Text'] = (df_clean['Skills'] + ' ' + 
                             df_clean['Job Description'] + ' ' + 
                             df_clean['Certifications'])

# Nettoyer le texte combiné
df_clean['Combined_Text'] = df_clean['Combined_Text'].apply(clean_text)

print("Feature combinée créée")
print("\nExemple de texte combiné:")
print(df_clean['Combined_Text'].iloc[0])

Feature combinée créée

Exemple de texte combiné:
java spring boot mysql git rest apis conçoit et développe des applications backend fiables en respectant les standards de performance et de securite oracle certified professional java se aws certified developer


## Séparation des données

In [14]:
# Préparer X (features) et y (target)
X = df_clean['Combined_Text']
y = df_clean['Job Title Encoded']

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")

Shape de X: (2458,)
Shape de y: (2458,)


In [15]:
# Diviser les données en ensembles d'entraînement et de test
# 80% train, 20% test avec stratification pour équilibrer les classes
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=None
)

print(f"Taille de l'ensemble d'entraînement: {len(X_train)}")
print(f"Taille de l'ensemble de test: {len(X_test)}")
print(f"\nDistribution dans l'ensemble d'entraînement:")
print(y_train.value_counts().head(10))

Taille de l'ensemble d'entraînement: 1966
Taille de l'ensemble de test: 492

Distribution dans l'ensemble d'entraînement:
Job Title Encoded
62     40
101    40
31     37
89     37
95     37
27     37
0      36
67     36
100    36
46     35
Name: count, dtype: int64


## Sauvegarde des données preprocessed

In [ ]:
# Sauvegarder les données preprocessed
preprocessed_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'label_encoder': label_encoder,
    'df_clean': df_clean
}

with open('../data/pkl/preprocessed_data.pkl', 'wb') as f:
    pickle.dump(preprocessed_data, f)

print("Données preprocessed sauvegardées: ../data/pkl/preprocessed_data.pkl")

Données preprocessed sauvegardées: preprocessed_data.pkl


In [ ]:
print("="*60)
print("RÉSUMÉ DU PREPROCESSING")
print("="*60)
print(f"\n1. Données chargées: {df.shape[0]} lignes, {df.shape[1]} colonnes")
print(f"2. Données après nettoyage: {df_clean.shape[0]} lignes")
print(f"3. Nombre de classes: {len(label_encoder.classes_)}")
print(f"4. Ensemble d'entraînement: {len(X_train)} échantillons")
print(f"5. Ensemble de test: {len(X_test)} échantillons")
print(f"\n6. Features créées:")
print(f"   - Combined_Text: combinaison de Skills, Job Description, Certifications")
print(f"   - Job Title Encoded: variable cible encodée")
print(f"\n7. Fichiers sauvegardés:")
print(f"   - ../data/pkl/preprocessed_data.pkl")
print(f"   - ../data/pkl/label_encoder.pkl")
print("\n" + "="*60)
print("PREPROCESSING TERMINÉ AVEC SUCCÈS")
print("="*60)

RÉSUMÉ DU PREPROCESSING

1. Données chargées: 2458 lignes, 4 colonnes
2. Données après nettoyage: 2458 lignes
3. Nombre de classes: 119
4. Ensemble d'entraînement: 1966 échantillons
5. Ensemble de test: 492 échantillons

6. Features créées:
   - Combined_Text: combinaison de Skills, Job Description, Certifications
   - Job Title Encoded: variable cible encodée

7. Fichiers sauvegardés:
   - pkl/preprocessed_data.pkl
   - pkl/label_encoder.pkl

PREPROCESSING TERMINÉ AVEC SUCCÈS
